In [9]:
import pandas as pd
import numpy as np
import joblib 
import pickle as pk
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


df1 = pd.read_csv('../data/Symptom-severity.csv')
df1['Symptom'] = df1['Symptom'].str.replace('_',' ')

discrp = pd.read_csv("../data/symptom_Description.csv")
ektra7at = pd.read_csv("../data/symptom_precaution.csv")
df = pd.read_csv('../data/dataset.csv')
for col in df.columns:
    df[col] = df[col].str.replace('_',' ')

columns_to_drop = ['Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 'Symptom_11',
                    'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15', 'Symptom_16', 'Symptom_17']

df = df.drop(columns=columns_to_drop)
cols = df.columns
data = df[cols].values.flatten()

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)
df = df.fillna(0)
vals = df.values
symptoms = df1['Symptom'].unique()

for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = df1[df1['Symptom'] == symptoms[i]]['weight'].values[0]
    
d = pd.DataFrame(vals, columns=cols)
d = d.replace('dischromic  patches', 0)
d = d.replace('spotting  urination',0)
df = d.replace('foul smell of urine',0)

data = df.iloc[:,1:].values
labels = df['Disease'].values
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8,random_state=42)
rnd_forest = RandomForestClassifier(random_state=42, max_features='sqrt', n_estimators= 500, max_depth=13)
rnd_forest.fit(x_train,y_train)

RandomForestClassifier(max_depth=13, n_estimators=500, random_state=42)

In [3]:
with open("symptoms.pkl","wb") as f:
    pk.dump(rnd_forest,f)

In [2]:
import pickle

# Load the model from the pickle file
with open('../models/symptoms.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [20]:
import numpy as np
def predd(loaded_model, S1, S2, S3, S4, S5, S6):
    psymptoms = [S1, S2, S3, S4, S5, S6]
    selected_symptoms = []
    a = np.array(df1["Symptom"])
    b = np.array(df1["weight"])
    for j in range(len(psymptoms)):
        for k in range(len(a)):
            if psymptoms[j] == a[k]:
                selected_symptoms.append(a[k])
                psymptoms[j] = b[k]
    psy = [psymptoms]
    # Get prediction and decision function scores
    pred2 = loaded_model.predict(psy)
    decision_function_scores = loaded_model.predict_proba(psy)
    
    # Get the index of the predicted class
    predicted_class_index = np.argmax(decision_function_scores)
    confidence_percentage = decision_function_scores[0, predicted_class_index] * 100
    # Additional code for getting disease description and recommendations
    disp = discrp[discrp['Disease'] == pred2[0]]
    disp = disp.values[0][1]
    recomnd = ektra7at[ektra7at['Disease'] == pred2[0]]
    c = np.where(ektra7at['Disease'] == pred2[0])[0][0]
    precaution_list = []
    for i in range(1, len(ektra7at.iloc[c])):
        precaution_list.append(ektra7at.iloc[c, i])

    print("The Disease Name: ", pred2[0])
    print("The Disease Description: ", disp)
    print("Recommended Things to do at home: ")
    for i in precaution_list:
        print(i)
    
    return pred2[0], confidence_percentage


In [21]:
predd(loaded_model,'itching', 'skin rash', 'nodal skin eruptions',0,0,0)

['itching', 'skin rash', 'nodal skin eruptions']
The Disease Name:  Fungal infection
The Disease Description:  In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, water, and plants. There are also some fungi that live naturally in the human body. Like many microbes, there are helpful fungi and harmful fungi.
Recommended Things to do at home: 
bath twice
use detol or neem in bathing water
keep infected area dry
use clean cloths


('Fungal infection', 100.0)

In [22]:
result, confidence_scores = predd(rnd_forest,'continuous sneezing', 'skin rash', 'nodal skin eruptions',0,0,0)

The Disease Name:  Allergy
The Disease Description:  An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens.
Recommended Things to do at home: 
apply calamine
cover area with bandage
nan
use ice to compress itching


In [23]:
print("Confidence Score: {:.2f}%".format(confidence_scores * 100))

Confidence Scores:  87.93602275020105
